In [86]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
import torchvision.transforms as transforms
from visualization.view_result import ray_shooting
import numpy as np
import torch
import matplotlib.pyplot as plt
import glob
import random
import os

# Import the UNet class
from visualization.view_result import ray_shooting


In [87]:
# Specify the device
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
   device = 'mps'

# Load the model and map it to the GPU
model = torch.load("model_full_old.pth", map_location=device)

# Set the model to evaluation mode
model.eval()

print("Model loaded onto", device)

/var/folders/5m/5hqxry713v946xwvr21xjx180000gn/T/ipykernel_26201/1405344667.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("model_full_old.pth", map_

Model loaded onto mps


In [88]:
def number_to_border( x, length, height):
    if x <= length:
        return (0, x)  #Top border
    elif x <= length + height:
        return (x - length, length - 1)  # Right border
    elif x <= 2 * length + height:
        return (height - 1, 2 * length + height - x)  # Bottom border
    else:
        return (2 * (length + height) - x, 0)  # Left border

In [89]:
num_runs = 15
image_files = glob.glob('./data_new/*.png')

for run in range(num_runs):
    # Randomly select one image
    target_image_path = random.choice(image_files)
    target_image = Image.open(target_image_path).convert('L')
    target_image = np.array(target_image)
    # Save target image to plots directory
    os.makedirs(f"./plots/{run}", exist_ok=True)
    plt.imsave(f"./plots/{run}/target_image.png", target_image, cmap='gray')

    # Random agent simulation
    # Assuming target_image is a numpy array or similar object
    length, height = target_image.shape  # .shape is an attribute, not a method
    
    num_samples = 15
    i=0
    
    # Create a black image
    image_black = np.zeros((length, height), dtype=np.uint8)
    
    while (i<num_samples):
        # Generate random border and angle for ray shooting
        border = np.random.randint(2 * (length + height - 1))
        angle = np.random.randint(360)
    
        border = number_to_border(border, 224, 224)
        
        # Perform ray shooting
        x, y = ray_shooting(border, angle, length, height, target_image)
        
        if x is None or y is None:
            continue
        elif x is not None and y is not None:
            image_black[x][y] = 1
    
        print(f"Selected Border Point (x, y): {border}")
        print(f"Ray Shooting Angle: {angle} degrees")
    
        print(f"Ray Shooting Result: (x, y) = ({x}, {y})")
        print(f"Angle for Ray: {angle} degrees")
        # Convert input array to a tensor suitable for the model
        input_tensor = torch.tensor(image_black, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
        input_tensor = input_tensor.unsqueeze(1)  # Add channel dimension
        input_tensor = input_tensor.to(device)  # Move the tensor to the GPU
    
        # Evaluate the model
        with torch.no_grad():  # Disable gradient calculation for evaluation
            output = model(input_tensor)  # Get the model's predictions
    
        # Convert the model output to a probability map and binary mask
        output_image = output[0][0].cpu().numpy()  # Get the first output channel as a numpy array
        binary_output = (output_image > 0.5).astype(np.uint8)  # Thresholding to create a binary mask
    
        # Print shape of the output image and binary output for clarity
        print(f"Output Image Shape: {output_image.shape}")
        print(f"Binary Mask Shape: {binary_output.shape}")
    
        # Visualization using Matplotlib
        plt.figure(figsize=(20, 5))
    
        
    
        found_points = np.where(image_black == 1)
        plt.subplot(1, 5, 1)
        plt.imshow(binary_output, cmap='gray')
        plt.scatter(found_points[1], found_points[0], color='green', s=10, label='Found Points')
        if x is not None and y is not None:
            plt.scatter(y, x, color='red', s=25, label='Current Point')
        plt.title('Binary Output with Points')
        plt.axis('off')
        # Add an arrow for the angle
        arrow_length = 20  # Adjust the length of the arrow
        arrow_dx = arrow_length * np.cos(np.radians(angle))  # Calculate arrow x component
        arrow_dy = arrow_length * np.sin(np.radians(angle))  # Calculate arrow y component
        plt.arrow(border[1], border[0], arrow_dy, arrow_dx, head_width=5, head_length=10, fc='red', ec='red', label='Angle')
    
        # Probability heatmap
        plt.subplot(1, 5, 2)
        plt.imshow(output_image, cmap='jet')
        plt.title('Probability Heatmap')
        plt.axis('off')
    
        plt.subplot(1, 5, 3)
        plt.imshow(target_image, cmap='gray')
        plt.title('Target Image')
        plt.axis('off')

        #Save the plot to /plots
        plt.savefig(f"./plots/{run}/{i}.png", bbox_inches='tight', dpi=100)
        plt.close()

        i = i+1

Selected Border Point (x, y): (0, 112)
Ray Shooting Angle: 337 degrees
Ray Shooting Result: (x, y) = (69, 83)
Angle for Ray: 337 degrees
Output Image Shape: (224, 224)
Binary Mask Shape: (224, 224)
Selected Border Point (x, y): (164, 0)
Ray Shooting Angle: 106 degrees
Ray Shooting Result: (x, y) = (144, 71)
Angle for Ray: 106 degrees
Output Image Shape: (224, 224)
Binary Mask Shape: (224, 224)
Selected Border Point (x, y): (92, 223)
Ray Shooting Angle: 284 degrees
Ray Shooting Result: (x, y) = (109, 153)
Angle for Ray: 284 degrees
Output Image Shape: (224, 224)
Binary Mask Shape: (224, 224)
Selected Border Point (x, y): (223, 207)
Ray Shooting Angle: 239 degrees
Ray Shooting Result: (x, y) = (171, 121)
Angle for Ray: 239 degrees
Output Image Shape: (224, 224)
Binary Mask Shape: (224, 224)
Selected Border Point (x, y): (91, 223)
Ray Shooting Angle: 295 degrees
Ray Shooting Result: (x, y) = (122, 156)
Angle for Ray: 295 degrees
Output Image Shape: (224, 224)
Binary Mask Shape: (224, 224)